In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import string
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from pylab import rcParams
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from stoch_gradient_descent import StochasticGradientDescent

%matplotlib inline

sns.set(style='whitegrid', palette='muted')

rcParams['figure.figsize'] = 14, 10
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

Here we read the user artist file and artist file, for artist we only want the id and name of the artist

In [2]:
plays = pd.read_csv('./data/user_artists.dat', sep='\t')
artists = pd.read_csv('./data/artists.dat', sep='\t', usecols=['id', 'name'])

In [3]:
plays.head(10)

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983
5,2,56,6152
6,2,57,5955
7,2,58,4616
8,2,59,4337
9,2,60,4147


In [4]:
artists.head(10)

,id,name
0,1,MALICE MIZER
1,2,Diary of Dreams
2,3,Carpathian Forest
3,4,Moi dix Mois
4,5,Bella Morte
5,6,Moonspell
6,7,Marilyn Manson
7,8,DIR EN GREY
8,9,Combichrist
9,10,Grendel


Merging the two dataframes to get the artist rakings based on number of plays

In [5]:
ap = pd.merge(artists, plays, how='inner', left_on='id', right_on='artistID')
ap = ap.rename(columns={"weight": 'totalPlay'})
ap.head(10)

,id,name,userID,artistID,totalPlay
0,1,MALICE MIZER,34,1,212
1,1,MALICE MIZER,274,1,483
2,1,MALICE MIZER,785,1,76
3,2,Diary of Dreams,135,2,1021
4,2,Diary of Dreams,257,2,152
5,2,Diary of Dreams,325,2,3466
6,2,Diary of Dreams,397,2,56
7,2,Diary of Dreams,560,2,134
8,2,Diary of Dreams,580,2,803
9,2,Diary of Dreams,935,2,428


In [6]:
artist_rank = ap.groupby(['name'])\
              .agg({'userID': 'count', 'totalPlay': 'sum'})\
              .rename(columns={"userID": 'totalUniqueUsers', 'totalPlay': 'totalNumberOfTimesPlayed'})\
              .sort_values(['totalNumberOfTimesPlayed'], ascending=False)
artist_rank['averageUserPlayed'] = artist_rank['totalNumberOfTimesPlayed'] / artist_rank['totalUniqueUsers']
artist_rank.head(20)

,totalUniqueUsers,totalNumberOfTimesPlayed,averageUserPlayed
name,,,
Britney Spears,522,2393140,4584.559387
Depeche Mode,282,1301308,4614.567376
Lady Gaga,611,1291387,2113.563011
Christina Aguilera,407,1058405,2600.503686
Paramore,399,963449,2414.659148
Madonna,429,921198,2147.314685
Rihanna,484,905423,1870.708678
Shakira,319,688529,2158.398119
The Beatles,480,662116,1379.408333


In [7]:
ap = ap.join(artist_rank, on='name', how='inner').sort_values('totalPlay', ascending=False)
ap.head(10)

,id,name,userID,artistID,totalPlay,totalUniqueUsers,totalNumberOfTimesPlayed,averageUserPlayed
2800,72,Depeche Mode,1642,72,352698,282,1301308,4614.567376
35843,792,Thalía,2071,792,324663,26,350035,13462.884615
27302,511,U2,1094,511,320725,185,493024,2664.994595
8152,203,Blur,1905,203,257978,114,318221,2791.412281
26670,498,Paramore,1664,498,227829,399,963449,2414.659148
21191,378,Evanescence,1146,378,203165,226,513476,2272.017699
14165,292,Christina Aguilera,514,292,176133,407,1058405,2600.503686
33309,701,Shakira,1983,701,172496,319,688529,2158.398119
33127,701,Shakira,757,701,169596,319,688529,2158.398119
31835,679,Glee Cast,1086,679,165902,249,506453,2033.947791


We are using collaborative filtering (CF) in order to built this recommender, the good thing about CF is that it has the capability to feature learning, which means it has the capability to learn which features to use.

In [8]:
playC = ap['totalPlay']

total_play_scaled = (playC - playC.min()) / (playC.max() - playC.min())

ap = ap.assign(totalPlayScaled=total_play_scaled)
ap.head(10)

,id,name,userID,artistID,totalPlay,totalUniqueUsers,totalNumberOfTimesPlayed,averageUserPlayed,totalPlayScaled
2800,72,Depeche Mode,1642,72,352698,282,1301308,4614.567376,1.000000
35843,792,Thalía,2071,792,324663,26,350035,13462.884615,0.920513
27302,511,U2,1094,511,320725,185,493024,2664.994595,0.909347
8152,203,Blur,1905,203,257978,114,318221,2791.412281,0.731441
26670,498,Paramore,1664,498,227829,399,963449,2414.659148,0.645960
21191,378,Evanescence,1146,378,203165,226,513476,2272.017699,0.576030
14165,292,Christina Aguilera,514,292,176133,407,1058405,2600.503686,0.499386
33309,701,Shakira,1983,701,172496,319,688529,2158.398119,0.489074
33127,701,Shakira,757,701,169596,319,688529,2158.398119,0.480852
31835,679,Glee Cast,1086,679,165902,249,506453,2033.947791,0.470378


In [9]:
ratings_df = ap.pivot(index='userID', columns='artistID', values='totalPlayScaled')
ratings_df.head(10)

artistID,1,2,3,4,5,6,7,8,9,10,...,18736,18737,18738,18739,18740,18741,18742,18743,18744,18745
userID,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001661,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
ratings = ratings_df.fillna(0).values
ratings

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00115397, ..., 0.        , 0.        ,
        0.        ]])

In [11]:
MIN_USER_RATINGS = 35
DELETE_RATING_COUNT = 15

def train_test_split(ratings):
    validation = np.zeros(ratings.shape)
    train = ratings.copy()
    print(validation)
    print(train)
    
    for user in np.arange(ratings.shape[0]):
        if len(ratings[user, :].nonzero()[0]) >= MIN_USER_RATINGS: # Minimum user need to have 35 and more ratings for artist played
            val_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                           size=DELETE_RATING_COUNT, replace=False)
            
            train[user, val_ratings] = 0
            validation[user, val_ratings] = ratings[user, val_ratings]
    
    return train, validation
            
            
train, validation = train_test_split(ratings)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.00115397 ... 0.         0.         0.        ]]


In [17]:
train.shape

(1892, 17632)

In [18]:
validation.shape

(1892, 17632)

In [28]:
P = 3 * np.random.rand(3, m)

In [21]:
m, n = train.shape

In [22]:
m

1892

In [23]:
n

17632

In [29]:
Q = 3 * np.random.rand(3, n)

In [26]:
users, items = train.nonzero()

In [27]:
for u, i in zip(users, items):
    print(u, i)

0 45
0 46
0 47
0 50
0 51
0 52
0 54
0 55
0 56
0 59
0 60
0 61
0 63
0 65
0 66
0 67
0 68
0 69
0 72
0 73
0 74
0 76
0 78
0 79
0 80
0 81
0 83
0 85
0 86
0 87
0 88
0 89
0 91
0 92
0 94
1 96
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 114
1 115
1 118
1 119
1 120
1 121
1 122
1 123
1 126
1 127
1 129
1 130
1 131
1 132
1 135
1 136
1 137
1 138
1 142
1 143
2 45
2 58
2 64
2 71
2 145
2 146
2 149
2 151
2 152
2 154
2 158
2 159
2 160
2 161
2 162
2 163
2 164
2 165
2 168
2 169
2 170
2 171
2 172
2 173
2 174
2 175
2 176
2 177
2 178
2 179
2 182
2 183
2 184
2 185
2 186
3 47
3 148
3 167
3 184
3 188
3 189
3 190
3 191
3 193
3 195
3 197
3 202
3 204
3 205
3 207
3 209
3 210
3 211
3 212
3 213
3 214
3 215
3 216
3 218
3 219
3 220
3 221
3 222
3 224
3 225
3 226
3 227
3 228
3 229
3 230
4 71
4 233
4 234
4 235
4 236
4 237
4 242
4 247
4 248
4 249
4 250
4 251
4 254
4 255
4 256
4 257
4 258
4 259
4 260
4 263
4 264
4 265
4 266
4 268
4 270
4 271
4 272
4 273
4 274
4 275
4 276
4 277
4 278
4 279


37 1324
37 1325
37 1326
38 700
38 834
38 938
38 945
38 948
38 1248
38 1252
38 1328
38 1329
38 1330
38 1331
38 1332
38 1333
38 1335
38 1337
38 1339
38 1340
38 1341
38 1342
38 1343
38 1344
38 1345
38 1346
38 1347
38 1348
38 1352
38 1353
38 1354
38 1358
38 1359
38 1360
38 1361
38 1362
38 1364
38 1365
39 148
39 167
39 178
39 184
39 185
39 198
39 215
39 219
39 222
39 223
39 282
39 285
39 313
39 371
39 372
39 466
39 528
39 683
39 957
39 1081
39 1366
39 1368
39 1371
39 1372
39 1373
39 1374
39 1375
39 1376
39 1377
39 1378
39 1379
39 1380
39 1381
39 1383
39 1384
40 50
40 52
40 58
40 59
40 74
40 78
40 82
40 148
40 221
40 224
40 232
40 272
40 300
40 322
40 323
40 325
40 371
40 480
40 597
40 739
40 748
40 898
40 1000
40 1089
40 1090
40 1385
40 1386
40 1387
40 1388
40 1389
40 1391
40 1392
40 1396
40 1400
40 1401
41 13
41 23
41 61
41 146
41 157
41 184
41 209
41 220
41 430
41 556
41 603
41 608
41 624
41 700
41 711
41 712
41 845
41 908
41 909
41 948
41 955
41 1110
41 1122
41 1207
41 1360
41 1402
41 14

76 1538
76 1852
76 2170
76 2172
76 2173
76 2174
76 2175
76 2176
76 2177
76 2178
76 2179
76 2180
76 2181
76 2182
76 2183
76 2184
76 2185
76 2186
76 2187
76 2189
76 2190
76 2191
76 2192
76 2193
76 2197
76 2198
76 2201
77 270
77 272
77 1964
77 2202
77 2203
77 2205
77 2206
77 2208
77 2209
77 2210
77 2212
77 2214
77 2215
77 2216
77 2219
77 2220
77 2221
77 2222
77 2223
77 2224
77 2225
77 2226
77 2228
77 2229
77 2232
77 2235
77 2236
77 2237
77 2238
77 2240
77 2242
77 2243
77 2244
77 2245
77 2247
78 413
78 420
78 427
78 439
78 616
78 1097
78 1108
78 1130
78 1150
78 1624
78 1728
78 2248
78 2249
78 2250
78 2251
78 2252
78 2253
78 2254
78 2255
78 2256
78 2257
78 2261
78 2262
78 2264
78 2265
78 2266
78 2267
78 2269
78 2271
78 2273
78 2275
78 2278
78 2279
78 2281
78 2282
79 148
79 190
79 194
79 950
79 1100
79 2283
79 2284
79 2285
79 2286
79 2287
79 2288
79 2289
79 2290
79 2292
79 2293
79 2296
79 2297
79 2298
79 2299
79 2302
79 2303
79 2305
79 2306
79 2308
79 2309
79 2311
79 2312
79 2314
79 2316
79 

113 825
113 1136
113 1341
113 1663
113 1788
113 1850
113 1861
113 1903
113 2047
113 2724
113 2920
113 2922
113 2923
113 2925
113 2926
113 2927
113 2928
113 2931
113 2934
113 2935
113 2936
113 2940
114 182
114 364
114 511
114 718
114 803
114 811
114 826
114 832
114 834
114 908
114 909
114 927
114 934
114 1101
114 1103
114 1104
114 1125
114 1734
114 1979
114 2196
114 2364
114 2634
114 2942
114 2943
114 2944
114 2946
114 2947
114 2948
114 2950
114 2951
114 2952
114 2953
114 2954
114 2955
114 2956
115 59
115 151
115 185
115 251
115 282
115 283
115 284
115 285
115 294
115 295
115 321
115 325
115 326
115 336
115 338
115 372
115 397
115 455
115 469
115 480
115 492
115 687
115 695
115 698
115 906
115 1023
115 1371
115 1461
115 1547
115 1919
115 2957
115 2958
115 2959
115 2960
115 2962
116 251
116 277
116 282
116 283
116 286
116 289
116 294
116 295
116 321
116 327
116 336
116 354
116 371
116 372
116 455
116 469
116 536
116 635
116 695
116 793
116 957
116 1033
116 1238
116 1420
116 1461
116 2065

149 866
149 1074
149 1197
149 1208
149 1218
149 1525
149 1694
149 1869
149 2466
149 3317
149 3393
149 3394
149 3395
149 3397
149 3399
149 3403
149 3404
149 3405
149 3406
149 3407
149 3409
149 3410
149 3411
149 3414
149 3416
149 3417
149 3418
149 3419
150 59
150 153
150 157
150 164
150 165
150 184
150 201
150 214
150 289
150 338
150 349
150 412
150 430
150 449
150 505
150 527
150 532
150 598
150 729
150 952
150 1039
150 1368
150 1797
150 2835
150 3037
150 3421
150 3422
150 3424
150 3425
150 3426
150 3428
150 3429
150 3430
150 3431
150 3432
151 157
151 170
151 222
151 460
151 624
151 700
151 706
151 859
151 908
151 909
151 919
151 938
151 1349
151 1351
151 1357
151 1360
151 1403
151 1630
151 1694
151 1798
151 1799
151 1801
151 2328
151 2332
151 2764
151 2767
151 2814
151 2953
151 3433
151 3437
151 3438
151 3439
151 3442
151 3443
151 3445
152 65
152 284
152 285
152 288
152 299
152 305
152 319
152 368
152 380
152 459
152 522
152 542
152 673
152 788
152 900
152 906
152 1073
152 1441
152 144

...

1713 338
1713 353
1713 460
1713 517
1713 536
1713 538
1713 668
1713 673
1713 680
1713 691
1713 695
1713 698
1713 781
1713 793
1713 899
1713 946
1713 1025
1713 6314
1713 9334
1713 9760
1713 10352
1714 6
1714 66
1714 83
1714 148
1714 167
1714 184
1714 203
1714 227
1714 282
1714 285
1714 286
1714 289
1714 292
1714 296
1714 310
1714 372
1714 419
1714 426
1714 430
1714 490
1714 493
1714 520
1714 531
1714 691
1714 695
1714 748
1714 834
1714 952
1714 1366
1714 2068
1714 2262
1714 2431
1714 2504
1714 3090
1714 4610
1715 45
1715 52
1715 53
1715 61
1715 66
1715 79
1715 197
1715 221
1715 423
1715 435
1715 614
1715 842
1715 843
1715 866
1715 977
1715 1008
1715 1074
1715 1802
1715 2918
1715 3181
1715 3242
1715 3250
1715 3259
1715 3399
1715 3865
1715 4401
1715 6635
1715 8354
1715 8455
1715 9105
1715 9236
1715 13039
1715 14617
1715 14860
1715 16532
1716 83
1716 282
1716 294
1716 368
1716 369
1716 373
1716 492
1716 493
1716 673
1716 1907
1716 2071
1716 2073
1716 2079
1716 2084
1716 2096
1716 2101
1716

1751 1053
1751 1063
1751 1704
1751 1823
1751 1827
1751 1881
1751 2472
1751 2515
1751 3244
1751 4224
1751 6597
1751 10337
1751 10393
1751 13028
1751 16810
1751 16812
1751 16815
1751 16816
1751 16818
1751 16819
1751 16821
1751 16822
1751 16823
1751 16824
1752 13
1752 157
1752 221
1752 608
1752 701
1752 723
1752 828
1752 909
1752 914
1752 944
1752 945
1752 954
1752 1118
1752 1123
1752 1513
1752 1660
1752 1769
1752 1959
1752 2364
1752 2549
1752 3580
1752 3643
1752 4008
1752 4193
1752 4579
1752 5039
1752 9938
1752 16825
1752 16826
1752 16827
1752 16828
1752 16829
1752 16830
1752 16831
1752 16832
1753 52
1753 59
1753 64
1753 185
1753 259
1753 273
1753 285
1753 289
1753 290
1753 294
1753 296
1753 320
1753 383
1753 400
1753 721
1753 946
1753 1089
1753 1370
1753 1547
1753 1679
1753 1763
1753 1824
1753 1839
1753 2029
1753 2036
1753 2039
1753 4052
1753 4063
1753 4460
1753 4963
1753 7005
1753 8643
1753 11933
1753 16833
1753 16834
1754 39
1754 58
1754 66
1754 82
1754 146
1754 184
1754 204
1754 219


1786 7372
1786 16993
1787 7
1787 371
1787 390
1787 492
1787 494
1787 498
1787 508
1787 531
1787 804
1787 807
1787 809
1787 821
1787 834
1787 908
1787 909
1787 911
1787 919
1787 921
1787 923
1787 948
1787 1330
1787 1519
1787 1900
1787 1911
1787 1913
1787 1918
1787 2047
1787 2128
1787 2153
1787 2830
1787 2920
1787 4408
1787 4966
1787 4987
1787 5838
1788 146
1788 157
1788 221
1788 228
1788 374
1788 492
1788 601
1788 700
1788 701
1788 718
1788 832
1788 908
1788 909
1788 919
1788 937
1788 944
1788 945
1788 1349
1788 1355
1788 1360
1788 1403
1788 1406
1788 1663
1788 2326
1788 2327
1788 2328
1788 2332
1788 2726
1788 2745
1788 5329
1788 5647
1788 7857
1788 8158
1788 8193
1788 16994
1789 66
1789 153
1789 157
1789 167
1789 184
1789 191
1789 194
1789 199
1789 217
1789 701
1789 842
1789 852
1789 872
1789 955
1789 1404
1789 1781
1789 1966
1789 2211
1789 2328
1789 2448
1789 3232
1789 3425
1789 3583
1789 3593
1789 3637
1789 4576
1789 5056
1789 5184
1789 6678
1789 6988
1789 9767
1789 13801
1789 16995


1825 430
1825 701
1825 855
1825 908
1825 992
1825 1005
1825 1010
1825 1122
1825 1321
1825 2168
1825 2332
1825 3252
1825 3603
1825 13745
1825 17241
1825 17242
1825 17243
1825 17244
1825 17245
1825 17246
1825 17248
1825 17249
1825 17251
1825 17252
1825 17253
1825 17254
1825 17255
1825 17256
1825 17257
1825 17258
1825 17260
1825 17261
1826 49
1826 155
1826 222
1826 286
1826 287
1826 294
1826 300
1826 305
1826 312
1826 318
1826 327
1826 458
1826 465
1826 492
1826 674
1826 1025
1826 1029
1826 1034
1826 1461
1826 2957
1826 3851
1826 4093
1826 4180
1826 4636
1826 5561
1826 7565
1826 15696
1826 15837
1826 17263
1826 17264
1826 17265
1826 17266
1826 17268
1826 17269
1826 17271
1827 232
1827 438
1827 597
1827 735
1827 748
1827 763
1827 1081
1827 1090
1827 1213
1827 1389
1827 1736
1827 1950
1827 2131
1827 2648
1827 2650
1827 2652
1827 2653
1827 2657
1827 2661
1827 3144
1827 4232
1827 4624
1827 4750
1827 4753
1827 4755
1827 6150
1827 8021
1827 10324
1827 10960
1827 12057
1827 15879
1827 17273
1827

1864 959
1864 963
1864 1415
1864 1581
1864 2007
1864 2914
1864 3452
1864 3687
1864 3814
1864 4170
1864 6287
1864 6375
1864 7819
1864 7836
1864 7857
1864 8097
1864 9815
1864 10352
1864 11198
1864 13982
1864 14515
1865 91
1865 184
1865 201
1865 203
1865 227
1865 310
1865 356
1865 422
1865 447
1865 469
1865 480
1865 493
1865 534
1865 674
1865 683
1865 1025
1865 1034
1865 1035
1865 1171
1865 1222
1865 1237
1865 1366
1865 1663
1865 2509
1865 3054
1865 3061
1865 3600
1865 3677
1865 6069
1865 6072
1865 6171
1865 15939
1865 17463
1865 17464
1865 17465
1866 6
1866 170
1866 184
1866 214
1866 221
1866 223
1866 224
1866 327
1866 371
1866 372
1866 466
1866 480
1866 482
1866 492
1866 518
1866 530
1866 718
1866 770
1866 799
1866 807
1866 942
1866 973
1866 1082
1866 1177
1866 1397
1866 1403
1866 1446
1866 1664
1866 2331
1866 2768
1866 3296
1866 3659
1866 4225
1866 7176
1866 17466
1867 61
1867 83
1867 155
1867 251
1867 279
1867 286
1867 288
1867 289
1867 294
1867 295
1867 296
1867 304
1867 314
1867 323

In [40]:
P[:,u]

array([1.25292764, 0.92619936, 2.16032825])

In [33]:
Q[:, i]

array([1.48724813, 1.48906229, 1.77283039])

In [44]:
error = train[u, i] - np.dot(P[:,u].T, Q[:,i])

In [36]:
train[u,i]

0.0007428472598292585

In [45]:
error

-7.071735544795385

In [47]:
np.dot(P[:,u].T, Q[:,i])

7.072478392055214